# Example of adversarial attack on ResNet50 network

In [ ]:
# import necessary packages
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

### Load ResNet50 network with weights and set parameters

In [ ]:
model = ResNet50(weights="imagenet")
print('ResNet50 model loaded with',len(model.layers),'layers')
optimizer = Adam(learning_rate=0.01)
lossFunct = SparseCategoricalCrossentropy()

### Load the initial image

In [ ]:
image_name = 'kot-frusia'
#image_name = 'lama'

image = cv2.imread(f"{image_name}.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))
image = np.expand_dims(image, axis=0)
# change to tensor
baseImage = tf.constant(image, dtype=tf.float32)
print("Image",image_name,"loaded")

### Original prediction

In [ ]:
org_pred = model.predict(tf.keras.applications.resnet50.preprocess_input(image))
dec_pred = decode_predictions(org_pred, top=3)[0]
real_class = org_pred.argmax()
real_label = dec_pred[0][1]
real_pred = dec_pred[0][2]
print("Predicted class: ",real_class,real_label,real_pred)
plt.imshow(image[0])
plt.title(f"{real_class} {real_label}:{real_pred*100:.2f}%")
plt.axis("off")  
plt.show()

### One learning step

In [ ]:
def do_step(image,delta,real_class):
    with tf.GradientTape() as tape:
        tape.watch(delta)
        adversary = tf.keras.applications.resnet50.preprocess_input(baseImage + delta)
        predictions = model(adversary, training=False)
        originalLoss = lossFunct(tf.convert_to_tensor([real_class]),predictions)
        loss = - originalLoss
        
        originalLoss = lossFunct(tf.convert_to_tensor([real_class]),predictions)
        targetLoss = lossFunct(tf.convert_to_tensor([target_class]),predictions)
        loss = targetLoss - originalLoss  # goal: minimize error to targetLoss and maximize error to originalLoss

        
    gradients = tape.gradient(loss, delta)
    #optimizer.apply_gradients([(gradients, delta)])
    #delta.assign_add(delta)
    
    optimizer.apply_gradients([(gradients, delta)])
    clipped_delta = tf.clip_by_value(delta, clip_value_min=-0.01, clip_value_max=0.01)
    delta.assign_add(clipped_delta)

    
    return delta,loss

### Visualisation

In [ ]:
def show_step(delta,loss):  
    print("step: {}, loss: {}...".format(step, loss.numpy()))
    adverImage = (baseImage + delta).numpy().squeeze()
    adverImage = np.clip(adverImage, 0, 255).astype("uint8")
    adverImage = np.expand_dims(adverImage, axis=0)
    predictions = model.predict(tf.keras.applications.resnet50.preprocess_input(adverImage))
    top3 = decode_predictions(predictions, top=3)[0]
    print("{}. {} -> ({}:{:.2f}, {}:{:.2f}, {}:{:.2f})".format(step,real_label,
                top3[0][1],top3[0][2],
                top3[1][1],top3[1][2],
                top3[2][1],top3[2][2]))

    fig=plt.figure(figsize=(12,8))
    ax = plt.subplot(1, 3, 1)
    plt.title(f"Original {real_label}:{100*real_pred:.2f}%")
    plt.imshow(image.squeeze())
    plt.axis("off")  
    ax = plt.subplot(1, 3, 2)
    plt.title("Delta")
    plt.imshow(np.clip((delta.numpy().squeeze()*256).astype("uint8"), 0, 255))
    plt.axis("off")  
    ax = plt.subplot(1, 3, 3)
    plt.title(f"Adversary {top3[0][1]}:{100*top3[0][2]:.2f}%")
    plt.imshow(adverImage.squeeze())
    plt.axis("off")  
    plt.show() 
    cv2.imwrite(f"outdir/{image_name}_{step:02d}.jpg", cv2.cvtColor(adverImage.squeeze(), cv2.COLOR_RGB2BGR))


### The learning loop

In [ ]:
steps = 100

target_class = 355
delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)
for step in range(0, steps):
    delta,loss = do_step(baseImage, delta, real_class)
    if step % 2==0:
        show_step(delta,loss)
print("Done")